In [1]:
import os
import igl
import json
import numpy as np
import polyscope as ps
from meshplot import plot, subplot, interact

# Kelvinlets
Performs the deformation of a single point based on the regularized kelvinlets


In [2]:
# Some important directories
obj = "decimated-knight"
actions = ["scale", "pinch", "twist"]
DATA = "../data/"
SAVE = f"../results/_{obj}/"


In [3]:
# Make a results folder
if not os.path.exists(SAVE):
  
  # Create a new directory because it does not exist 
  os.makedirs(SAVE)
  for action in actions:
    os.makedirs(f"{SAVE}{action}/")
  print(f"Images will be saved to {SAVE}!")

In [4]:
# a few options
ps.set_program_name("Regularized Kelvinets")
ps.set_verbosity(2)
ps.set_use_prefs_file(False)

# enable auto centering and scaling
ps.set_autocenter_structures(True)
ps.set_autoscale_structures(True)

In [5]:
# Read in an object
v, f = igl.read_triangle_mesh(f"{DATA}{obj}.off")


In [6]:
# Read in the points we want to manipulate
selected = open(f'{DATA}{obj}-selected.json')
manipulate_points = json.load(selected)
manipulate_points

{'scale': 314, 'pinch': 314, 'twist': 314}

In [7]:
# Initialize polyscope
ps.init()

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 INTEL-14.7.28


In [8]:
action = "pinch"
#x0 = #np.array(v[manipulate_points[action], :])
#x0[2] = 1.
x0 = np.array([0., 0., 0.])
print(x0, x0.shape)

r = v - x0
r_norm = np.linalg.norm(r, ord=2, axis=1)
print(r.shape, r_norm.shape)

epsilon = 0.5
r_epsilon = np.sqrt(np.power(r_norm, 2) + epsilon**2)
r_epsilon = np.stack([r_epsilon, r_epsilon, r_epsilon], axis=1)
print(r_epsilon.shape)

F_scale = np.identity(3)
F_twist = np.array([[0., 1., -1.], [-1., 0., 1.], [1., -1., 0.]])
F_pinch = np.array([[0., 1., 1.], [1., 0., 1.], [1., 1., 0.]])

[0. 0. 0.] (3,)
(502, 3) (502,)
(502, 3)


In [9]:
POISSON_RATIO = 10
SHEAR_MODULUS = 0.11
a = 1 / (4 * np.pi * SHEAR_MODULUS)
b = a / (4 * (1 - POISSON_RATIO))
c = 2 / (3 * a - 2 * b)

In [10]:
# Register the mesh
mesh = ps.register_surface_mesh(f'{obj}',v,f)
#points = ps.register_point_cloud(f'pts', v)

In [11]:
# Save original look
t = 0
ps.screenshot(filename=f"{SAVE}{t:04d}.png")

## Deformation constants

### Do twist

In [12]:
t = 1

In [13]:
t_disp = -a*np.multiply(( (1/np.power(r_epsilon, 3)) + (1.5*epsilon**2/np.power(r_epsilon, 5)) ), ((F_twist @ r.T).T))
twist_v = 0.5*t_disp + v

In [14]:
# Add scaled points
#twist_pts = ps.register_point_cloud(f'twisted pts', twist_v)
twist_mesh = ps.register_surface_mesh(f'{obj} twist',twist_v,f)

In [15]:
ps.show()